In [12]:
#importing libraries
from pprint import pprint
import urllib.request
import json
from datetime import datetime
import datetime
import pandas as pd
import pymongo
import matplotlib.pyplot as plt
from pymongo import MongoClient

# Reading json File

In [13]:
#reading json file "Electric Consumption And Cost.json"

with open("Electric.json",'r') as elec_file:
    elec_data=json.load(elec_file)
    
#elec_data


In [3]:
elec_data['meta']['view'].keys()

dict_keys(['id', 'name', 'assetType', 'attribution', 'averageRating', 'category', 'createdAt', 'description', 'displayType', 'downloadCount', 'hideFromCatalog', 'hideFromDataJson', 'indexUpdatedAt', 'newBackend', 'numberOfComments', 'oid', 'provenance', 'publicationAppendEnabled', 'publicationDate', 'publicationGroup', 'publicationStage', 'rowClass', 'rowsUpdatedAt', 'rowsUpdatedBy', 'tableId', 'totalTimesRated', 'viewCount', 'viewLastModified', 'viewType', 'approvals', 'clientContext', 'columns', 'grants', 'metadata', 'owner', 'query', 'rights', 'tableAuthor', 'tags', 'flags'])

In [4]:
#Reading the column names from metadat of json file
elec_data['meta']['view']['columns']

print("Number of columns:",len(elec_data['meta']['view']['columns']))

num_columns = len(elec_data['meta']['view']['columns'][0].keys())


Number of columns: 35


In [5]:
d=[]
for i in range(len(elec_data['data'])):
    d.append(elec_data['data'][i][21])
#d

In [10]:
#try:
    for x in elec_data['data']:
        dt_col=x[21]
    
        #Converting the tect colum to date type in format "YYYY-MM-DD"
        year,month=map(int, dt_col.split("-"))
        date_obj = datetime.date(year=year, month=month, day=1)
        
        x[21] = date_obj.isoformat()
# except ValueError as ve:
#     print("Error")
        

IndentationError: unexpected indent (2263359072.py, line 2)

In [11]:
for x in elec_data['data']:
    dt_col=x[21]
    
    #Converting the tect colum to date type in format "YYYY-MM-DD"
    year,month=map(int, dt_col.split("-"))
    date_obj = datetime.date(year=year, month=month, day=1)
    
    x[21] = date_obj.isoformat()

ValueError: too many values to unpack (expected 2)

In [7]:
print("Rows:",len(elec_data['data']))    #rows
print("Columns:",len(elec_data['data'][0]))      #cols


Rows: 447849
Columns: 35


In [8]:
#Deleting rows with dates before 2013

elec_data['data'] = [row for row in elec_data['data'] if row[21] >= '2013-01-01']

#Deleting rows with dates after 2020
elec_data['data'] = [row for row in elec_data['data'] if row[21] < '2021-01-01']

# for i in elec_data['data']:
#     print(i[21])


In [9]:
print("Rows:",len(elec_data['data']))    #rows
print("Columns:",len(elec_data['data'][0]))      #cols




Rows: 291344
Columns: 35


In [10]:
#modifying the file with the updated data (after removing unwanted columns)
with open('Electric Consumption And Cost.json', 'w') as f:
    json.dump(elec_data, f)

In [11]:
#creating a array to store the columns name from json file
elec_col_nm=[]


for i in elec_data['meta']['view']['columns']:
    elec_col_nm.append(i['name'])
    
elec_col_nm

['sid',
 'id',
 'position',
 'created_at',
 'created_meta',
 'updated_at',
 'updated_meta',
 'meta',
 'Development Name',
 'Borough',
 'Account Name',
 'Location',
 'Meter AMR',
 'Meter Scope',
 'TDS #',
 'EDP',
 'RC Code',
 'Funding Source',
 'AMP #',
 'Vendor Name',
 'UMIS BILL ID',
 'Revenue Month',
 'Service Start Date',
 'Service End Date',
 '# days',
 'Meter Number',
 'Estimated',
 'Current Charges',
 'Rate Class',
 'Bill Analyzed',
 'Consumption (KWH)',
 'KWH Charges',
 'Consumption (KW)',
 'KW Charges',
 'Other charges']

In [49]:
#getting the data into dataframe
elec_df=pd.DataFrame(elec_data['data'],columns=elec_col_nm)

elec_df.head()


,sid,id,position,created_at,created_meta,updated_at,updated_meta,meta,Development Name,Borough,...,Meter Number,Estimated,Current Charges,Rate Class,Bill Analyzed,Consumption (KWH),KWH Charges,Consumption (KW),KW Charges,Other charges
0,row-imab-j6zy~3p2z,00000000-0000-0000-A4A4-42CAC0E87432,0,1571253526,None,1571253526,None,{ },ADAMS,BRONX,...,7223256,N,15273.86,GOV/NYC/068,Yes,117600,6451.54,208.00,3676.32,5146.00
1,row-wneq-6ezk~8r72,00000000-0000-0000-0BE7-12A877AE6A43,0,1571253526,None,1571253526,None,{ },ADAMS,BRONX,...,7223256,N,16078.55,GOV/NYC/068,Yes,122400,6714.86,208.00,3676.32,5687.37
2,row-cjsy.jskv~bstp,00000000-0000-0000-BC82-8D6AF0BF66A9,0,1571253526,None,1571253526,None,{ },ADAMS,BRONX,...,7223256,N,16492.80,GOV/NYC/068,Yes,108800,5611.90,216.00,3792.97,7087.93
3,row-wra7.a59m~e7xg,00000000-0000-0000-48A0-AB277B04BB53,0,1571253526,None,1571253526,None,{ },ADAMS,BRONX,...,7223256,N,13942.26,GOV/NYC/068,Yes,102400,5281.79,208.00,3792.96,4867.51
4,row-vu3f.gau5.zd68,00000000-0000-0000-2956-3E801C5A8E62,0,1571253526,None,1571253526,None,{ },ADAMS,BRONX,...,7223256,N,14018.84,GOV/NYC/068,Yes,95200,4910.42,200.00,3792.96,5315.46


In [37]:
#Checking if any date has wrong formating

try:
    pd.to_datetime(elec_df['Service End Date'], format='%Y-%m-%dT%H:%M:%S', errors='raise')
    # do something
except ValueError:
    print("Dateformat is not constant")
    pass

In [39]:
try:
    pd.to_datetime(elec_df['Service Start Date'], format='%Y-%m-%dT%H:%M:%S', errors='raise')
    # do something
except ValueError:
    print("Dateformat is not constant")
    pass

In [13]:
elec_df.shape

(291344, 35)

In [50]:
#removing unrequired columns
elec_df=elec_df.drop(['sid','id','position','created_at','created_meta','updated_at','updated_meta','meta'],axis=1)

elec_df.head()

,Development Name,Borough,Account Name,Location,Meter AMR,Meter Scope,TDS #,EDP,RC Code,Funding Source,...,Meter Number,Estimated,Current Charges,Rate Class,Bill Analyzed,Consumption (KWH),KWH Charges,Consumption (KW),KW Charges,Other charges
0,ADAMS,BRONX,ADAMS,BLD 05,NONE,BLD 01 to 07,118,248,B011800,FEDERAL,...,7223256,N,15273.86,GOV/NYC/068,Yes,117600,6451.54,208.00,3676.32,5146.00
1,ADAMS,BRONX,ADAMS,BLD 05,NONE,BLD 01 to 07,118,248,B011800,FEDERAL,...,7223256,N,16078.55,GOV/NYC/068,Yes,122400,6714.86,208.00,3676.32,5687.37
2,ADAMS,BRONX,ADAMS,BLD 05,NONE,BLD 01 to 07,118,248,B011800,FEDERAL,...,7223256,N,16492.80,GOV/NYC/068,Yes,108800,5611.90,216.00,3792.97,7087.93
3,ADAMS,BRONX,ADAMS,BLD 05,NONE,BLD 01 to 07,118,248,B011800,FEDERAL,...,7223256,N,13942.26,GOV/NYC/068,Yes,102400,5281.79,208.00,3792.96,4867.51
4,ADAMS,BRONX,ADAMS,BLD 05,NONE,BLD 01 to 07,118,248,B011800,FEDERAL,...,7223256,N,14018.84,GOV/NYC/068,Yes,95200,4910.42,200.00,3792.96,5315.46


In [15]:
elec_df.shape

(291344, 27)

In [16]:
#removing unrequired columns
elec_df=elec_df.drop(['Meter AMR','Meter Scope','TDS #','EDP','RC Code','Funding Source','AMP #','Estimated','Bill Analyzed','Other charges'],axis=1)

In [17]:
elec_df.shape

(291344, 17)

In [18]:
print(elec_df.head(10))

  Development Name Borough Account Name Location               Vendor Name  \
0            ADAMS   BRONX        ADAMS   BLD 05  NEW YORK POWER AUTHORITY   
1            ADAMS   BRONX        ADAMS   BLD 05  NEW YORK POWER AUTHORITY   
2            ADAMS   BRONX        ADAMS   BLD 05  NEW YORK POWER AUTHORITY   
3            ADAMS   BRONX        ADAMS   BLD 05  NEW YORK POWER AUTHORITY   
4            ADAMS   BRONX        ADAMS   BLD 05  NEW YORK POWER AUTHORITY   
5            ADAMS   BRONX        ADAMS   BLD 05  NEW YORK POWER AUTHORITY   
6            ADAMS   BRONX        ADAMS   BLD 05  NEW YORK POWER AUTHORITY   
7            ADAMS   BRONX        ADAMS   BLD 05  NEW YORK POWER AUTHORITY   
8            ADAMS   BRONX        ADAMS   BLD 05  NEW YORK POWER AUTHORITY   
9            ADAMS   BRONX        ADAMS   BLD 05  NEW YORK POWER AUTHORITY   

  UMIS BILL ID Revenue Month   Service Start Date     Service End Date # days  \
0      2600212    2013-01-01  2012-12-24T00:00:00  2013-01-2

In [19]:
#checking for null records in data
print(elec_df.isnull().sum())  


Development Name         0
Borough                  0
Account Name             0
Location              6807
Vendor Name              0
UMIS BILL ID             0
Revenue Month            0
Service Start Date       8
Service End Date         8
# days                   8
Meter Number             0
Current Charges          0
Rate Class               0
Consumption (KWH)        0
KWH Charges              0
Consumption (KW)         0
KW Charges               0
dtype: int64


In [20]:
#deleting the rows with null values
elec_df.dropna(inplace=True)

In [21]:
#checking for null records in data
print(elec_df.isnull().sum())  


Development Name      0
Borough               0
Account Name          0
Location              0
Vendor Name           0
UMIS BILL ID          0
Revenue Month         0
Service Start Date    0
Service End Date      0
# days                0
Meter Number          0
Current Charges       0
Rate Class            0
Consumption (KWH)     0
KWH Charges           0
Consumption (KW)      0
KW Charges            0
dtype: int64


In [22]:
# number of rows and columns in data
print(elec_df.shape)  

 # listing column names
print(elec_df.columns)

# summary  of the numerical columns
print(elec_df.describe())  

#data types and non-null values
print(elec_df.info())  


(284529, 17)
Index(['Development Name', 'Borough', 'Account Name', 'Location',
       'Vendor Name', 'UMIS BILL ID', 'Revenue Month', 'Service Start Date',
       'Service End Date', '# days', 'Meter Number', 'Current Charges',
       'Rate Class', 'Consumption (KWH)', 'KWH Charges', 'Consumption (KW)',
       'KW Charges'],
      dtype='object')
       Development Name   Borough            Account Name Location  \
count            284529    284529                  284529   284529   
unique              367         7                     356      495   
top            EDENWALD  BROOKLYN  BROWNSVILLE/VAN DYKE I   BLD 01   
freq               5984    109022                    6357    33462   

                     Vendor Name UMIS BILL ID Revenue Month  \
count                     284529       284529        284529   
unique                         4        85488            84   
top     NEW YORK POWER AUTHORITY      8888737    2020-04-01   
freq                      282477          136   

In [23]:
#Removing columns which has very less unique values   
                 #"Borough"  column has also less unique , but we wont remove it as it is place 
elec_df=elec_df.drop(['Vendor Name'],axis=1)

In [24]:
  # drop duplicate rows
print("Before Deleting duplicates",elec_df.shape)
elec_df.drop_duplicates(inplace=True)
print("After deleting duplicated",elec_df.shape)

Before Deleting duplicates (284529, 16)
After deleting duplicated (274489, 16)


# Inserting Records in Mongodb

In [26]:
from pymongo import MongoClient
client = MongoClient("mongodb://localhost:27017", username='dap',password='dap')
db = client['Database']

db1 = db['Electrical_Consumption']
#delete or drop collection
db1.drop()
collection = db['Electrical_Consumption']
elec_records = elec_df.to_dict('records')
collection.insert_many(elec_records)

In [28]:
#shows the list of databases in mongodb

for x in db.list_collection_names():
    print(x)

Water_Consumption
Electrical_Consumption
